# Install and import relevant packages and libraries

In [1]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.8 MB/s eta 0:00:00


In [2]:
import scipy.io
import scipy.sparse as sparse
from scipy.sparse import csgraph
import numpy as np
import argparse
import logging

#additional imports needed on top of what was given in original code
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import Reddit
from torch_geometric.datasets import CitationFull
from torch_geometric.datasets import HeterophilousGraphDataset
from torch_geometric.datasets import PPI
from torch_geometric.utils import to_networkx
import pandas as pd
import networkx as nx
import os

# Mount Google drive (for usage in Google colab)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Loading the adjacancy matrix

In [5]:
## Load matrix from graph when using networkX
def load_adjacency_matrix_from_graph(graph):
    # Create an adjacency matrix from the NetworkX graph
    adjacency_matrix = nx.adjacency_matrix(graph)
    return adjacency_matrix

#Deepwalk filter

In [6]:
def deepwalk_filter(evals, window):
    #used to regularize eigenvalues (not have too small, or too large values)
    for i in range(len(evals)):
        x = evals[i]
        evals[i] = 1. if x >= 1 else x*(1-x**window) / (1-x) / window
    evals = np.maximum(evals, 0)
    print("After filtering, max eigenvalue=%f, min eigenvalue=%f",
            np.max(evals), np.min(evals))
    return evals

# Graph Laplacian

In [7]:
def approximate_normalized_graph_laplacian(A, rank, which="LA"):
    n = A.shape[0]
    L, d_rt = csgraph.laplacian(A, normed=True, return_diag=True) #laplacian and diagonal
    # X = D^{-1/2} W D^{-1/2}
    X = sparse.identity(n) - L # I - L
    print("Eigen decomposition...")
    #evals, evecs = sparse.linalg.eigsh(X, rank,
    #        which=which, tol=1e-3, maxiter=300)
    evals, evecs = sparse.linalg.eigsh(X.toarray(), rank, which=which) #get eigenvalues and eigenvectors of X
    print("Maximum eigenvalue %f, minimum eigenvalue %f", np.max(evals), np.min(evals))
    print("Computing D^{-1/2}U..")
    D_rt_inv = sparse.diags(d_rt ** -1)
    D_rt_invU = D_rt_inv.dot(evecs) #multiply inverted matrix by eigenvectors of X
    return evals, D_rt_invU

# Approximate Deepwalk matrix

In [8]:
def approximate_deepwalk_matrix(evals, D_rt_invU, window, vol, b):
    evals = deepwalk_filter(evals, window=window) #deepwalk filter, regularize eigenvalues
    X = sparse.diags(np.sqrt(evals)).dot(D_rt_invU.T).T #sqrt(eigenvalues) * inverted matrix
    X = torch.as_tensor(X, dtype=torch.float32)

    # Define a PyTorch model
    class DeepWalkModel(torch.nn.Module):
        def forward(self, m):
            mmT = torch.matmul(m, m.t()) * (vol / b)
            return torch.log(torch.max(mmT, torch.tensor(1.0)))

    # Initialize the model
    model = DeepWalkModel()

    # Convert the input to PyTorch tensor
    X_tensor = torch.as_tensor(X, dtype=torch.float32) #make X a torch tensor

    try:
        # Forward pass through the model
        Y_tensor = model(X_tensor)

        # Convert the result back to NumPy array
        Y = Y_tensor.detach().numpy()

        logging.info("Computed DeepWalk matrix with %d non-zero elements",
                     np.count_nonzero(Y))

        final = sparse.csr_matrix(Y)

        return final

    except Exception as e:
        logging.error("Error during PyTorch forward pass: %s", str(e))
        raise

#SVD deepwalk matrix

In [9]:
def svd_deepwalk_matrix(X, dim):

    # Ensure dim is an integer
    dim = int(dim)

    if dim >= min(X.shape):
        print('shape of X: ', X.shape)
        print('X', X)
        raise ValueError("The specified value of dim is too large for the given matrix X.")

    u, s, v = sparse.linalg.svds(X, dim, return_singular_vectors="u") #get the SVD decomposition of X
    return sparse.diags(np.sqrt(s)).dot(u.T).T #return final value U_d sqrt(S_d) like in last step paper algorithm

#NetMF large

In [10]:
def netmf_large(args):

    #set A = adjacency matrix
    A = args.input

    vol = float(A.sum())
    evals, D_rt_invU = approximate_normalized_graph_laplacian(A, rank=args.rank, which="LA")

    deepwalk_matrix = approximate_deepwalk_matrix(evals, D_rt_invU,
                                                  window=args.window,
                                                  vol=vol, b=args.negative)

    deepwalk_embedding = svd_deepwalk_matrix(deepwalk_matrix, dim=args.dim)

    # Ensure the output directory exists
    output_dir = os.path.dirname(args.output)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    try:
        np.save(args.output, deepwalk_embedding, allow_pickle=False)
        print("Embedding saved successfully.")
    except Exception as e:
        print("Error saving embedding: %s", str(e))


#Compute deepwalk matrix using pytorch

In [11]:
def direct_compute_deepwalk_matrix(A, window, b):
    #used with small window size only, based on formula given in paper
    n = A.shape[0]
    vol = float(A.sum())
    L, d_rt = csgraph.laplacian(A, normed=True, return_diag=True)
    # X = D^{-1/2} A D^{-1/2}
    X = sparse.identity(n) - L
    S = np.zeros_like(X)
    X_power = sparse.identity(n)


    #for loop calculates P^r in the sum of the window
    for i in range(window):
        print("Compute matrix %d-th power", i+1)
        #matrix computations become to computationally extreme here
        X_power = X_power.dot(X)
        S += X_power

    S *= vol / window / b #division (in beginnng of formula) * P^r = S
    D_rt_inv = sparse.diags(d_rt ** -1) #D^-1 in paper
    M = D_rt_inv.dot(D_rt_inv.dot(S).T) #(division*P^r) times D^-1

    print('before torch.tensor')

    # Convert matrices to PyTorch tensors
    M_tensor = torch.tensor(M.toarray(), dtype=torch.float32, requires_grad=False)

    print('after torch.tensor')
    # Define a PyTorch model
    class DirectComputeModel(torch.nn.Module):
        def forward(self, m):
            return torch.log(torch.max(m, torch.tensor(1.0)))

    # Initialize the model
    model = DirectComputeModel()

    # Forward pass through the model
    Y_tensor = model(M_tensor)

    # Convert the result back to a NumPy array
    Y = Y_tensor.detach().numpy()

    print("Computed DeepWalk matrix with %d non-zero elements",
                 np.count_nonzero(Y))

    return sparse.csr_matrix(Y)


#NetMF small

In [12]:
def netmf_small(args):
    print("Running NetMF for a small window size...")
    print("Window size is set to be ", args.window)
    # load adjacency matrix
    A = args.input

    # directly compute deepwalk matrix
    deepwalk_matrix = direct_compute_deepwalk_matrix(A,
            window=args.window, b=args.negative) #calculate deepwalk representation matrix, as defined in paper (step 2)

    # factorize deepwalk matrix with SVD
    deepwalk_embedding = svd_deepwalk_matrix(deepwalk_matrix, dim=args.dim) #create embedding (steps 4 and 5)
    print("Save embedding to ", args.output)
    np.save(args.output, deepwalk_embedding, allow_pickle=False)

In [13]:
class NetMFArgs:
    def __init__(self, input_matrix, output_file, rank, dim, window, negative):
        self.input = input_matrix
        self.output = output_file
        self.rank = rank
        self.dim = dim
        self.window = window
        self.negative = negative

# Usage for .csv files

In [15]:
# # Set parameters directly
# folder_path = "/content/drive/MyDrive/NetMF_implementations/Flickr-dataset/"

# nodes_file = folder_path + "data/nodes.csv"
# edges_file = folder_path + "data/edges.csv"
# groups_file = folder_path + "data/groups.csv"
# group_edges_file = folder_path + "data/group-edges.csv"
# output_file = "/content/drive/MyDrive/NetMF_implementations/output_flickr_large"

# nodes_id = pd.read_csv(nodes_file, header=None, names=['id'])
# groups_id = pd.read_csv(groups_file, header=None, names=['group'])
# edges = pd.read_csv(edges_file, header=None, names=['id_1', 'id_2'])
# user_group_membership = pd.read_csv(group_edges_file, header=None, names=['id', 'group'])


# # Define parameters (as in paper)
# rank = 16384 #static, 256 for all datasets except flickr, there 16384
# dim = 200 #subject to change
# window = 10 #static from paper
# negative = 1.0 #static from paper
# use_large_window = False  # Set to True for large window, False for small window

# # Create graph and add nodes and edges from it
# G_YT = nx.Graph()
# G_YT.add_nodes_from(nodes_id['id'])
# G_YT.add_edges_from(edges[['id_1', 'id_2']].values)

# # Load adjacency matrix from CSV files
# adjacency_matrix = load_adjacency_matrix_from_graph(G_YT)

# #Create NetMFArgs instance
# netmf_args = NetMFArgs(input_matrix=adjacency_matrix, output_file=output_file,
#                        rank=rank, dim=dim, window=window, negative=negative)

# # Directly call the relevant function with the NetMFArgs instance
# if use_large_window:
#     netmf_large(netmf_args)
# else:
#     netmf_small(netmf_args)

# Usage for pytorch_geometric datasets

In [16]:
#Set output file
output_file = "/content/drive/MyDrive/NetMF_implementations/output_ppi_large"

# Define parameters (as in paper)
rank = 256 #static, 256 for all datasets except flickr
dim = 200 #subject to change
window = 10 #static from paper
negative = 1.0 #static from paper
use_large_window = True  # Set to True for large window, False for small window

# Load the PubMed dataset
dataset = PPI("./")
data = dataset[0]
G = to_networkx(data, to_undirected=True)

# Load adjacency matrix from CSV files
adjacency_matrix = load_adjacency_matrix_from_graph(G)

#Create NetMFArgs instance
netmf_args = NetMFArgs(input_matrix=adjacency_matrix, output_file=output_file,
                       rank=rank, dim=dim, window=window, negative=negative)

# Directly call the relevant function with the NetMFArgs instance
if use_large_window:
    netmf_large(netmf_args)
else:
    netmf_small(netmf_args)

Extracting ./ppi.zip
Processing...
Done!


Eigen decomposition...
Maximum eigenvalue %f, minimum eigenvalue %f 1.000000000000022 0.46009476827894114
Computing D^{-1/2}U..
After filtering, max eigenvalue=%f, min eigenvalue=%f 1.0 0.0851814657737541
Embedding saved successfully.
